In [11]:
import pandas as pd
import numpy as np
import datetime
import random

typ_dict = {0: "UNCLASSIFIED", 1: "COMPLAINT", 2: "RECLAMATION",
            3: "INFORMATION", 4: "INVOICE COMPLAINT", 5: "INTERNAL ERROR",
            6: "SUGGESTION FOR IMPROVEMENT", 7: "RETURN CALL"}

quality_level_dict = {0: "HIGH", 1: "MEDIUM", 2: "LOW"}

process_status_dict = {1: "UNCLASSIFIED", 2: "RECEIVED", 3: "REJECTED",
                       4: "ASSIGNMENT", 5: "REVIEW", 6: "PROCESSED",
                       7: "REPLIED", 8: "COMPLETED", 9: "BINNED",
                       10: "DELETED", 12: "EXTERNAL_SYSTEM"}

department_dict = {1: "Customer service", 4: "Claims management", 105: "Invoice Complaints North"}

transitions = ["Received unclassified", "handover", "classify", "assign", "forward", "reject", "complete"]

def p_values(n):
    p = [np.random.choice(range(0, 5))]
    for i in range(1, n):
        p.append(np.random.choice(range(50, 100)))
    p = p / sum(p)
    return p

def gen_process(ID):
    character_start = customers[ID][1]
    character_middle = customers[ID][2]
    character_end = customers[ID][3]
    process = [1]
    i = 0
    while process[i] < 8:
        if process[i] == 1:
            choices = [1, 2]
            character = character_start
        elif process[i] == 6:
            choices = [4, 5, 6, 7, 8, 9, 10]
            character = character_end
        else:
            choices = [process[i]-1, process[i], process[i]+1]
            character = character_middle
        process.append(np.random.choice(choices, p = character))
        i += 1
    return process

def gen_customer(ID):
    customer = [ID]
    for i in [2, 3, 7]:
        customer.append(p_values(i))
    return customer

def gen_transitions(n):
    transition = [transitions[0]]
    for i in range(1, n-1):
        choice = np.random.choice(range(1, 6), p = [0.6, 0.1, 0.1, 0.1, 0.1])
        transition.append(transitions[choice])
    transition.append(transitions[6])
    return transition

num_customers = 100
customers = [gen_customer(i) for i in range(num_customers)]
calls_per_customer = np.random.poisson(lam = 6, size = num_customers)

draw_from = calls_per_customer.copy()

order = []
while sum(draw_from) > 0:
    pick_from = np.random.choice(np.array(range(len(draw_from)))[draw_from != 0])
    draw_from[pick_from] -= 1
    order.append(pick_from)

ORDER_STATUS_process_status = []
ORDER_STATUS_transition = []
for i in order:
    process_i = gen_process(i)
    ORDER_STATUS_process_status.append(process_i)
    ORDER_STATUS_transition.append(gen_transitions(len(process_i)))

# Vehicles
vehicles = list(range(25))
vehicle_quality_p = [p_values(3) for i in range(25)]
vehicle_list = [np.random.choice(vehicles) for i in range(len(order))]
vehicle_license_plate_column = []
quality_list = [np.random.choice([1, 2, 3], p = vehicle_quality_p[vehicle_list[i]]) for i in range(len(order))]
quality_level_column = []


process_status_column = []
transition_column = []
customer_contact_id_column = []
type_p = p_values(8)
typ_list = [np.random.choice(list(typ_dict.values()), p = type_p) for i in range(len(order))]
typ_column = []
priority_list = [np.random.choice([1, 2, 3], p = customers[order[i]][2]) for i in range(len(order))]
priority_column = []
service_order_id_column = []
for i in range(len(ORDER_STATUS_process_status)):
    for j in range(len(ORDER_STATUS_process_status[i])):
        process_status_column.append(ORDER_STATUS_process_status[i][j])
        transition_column.append(ORDER_STATUS_transition[i][j])
        customer_contact_id_column.append(order[i])
        typ_column.append(typ_list[i])
        priority_column.append(priority_list[i])
        service_order_id_column.append(i)
        vehicle_license_plate_column.append(vehicle_list[i])
        quality_level_column.append(quality_list[i])
process_status_column = np.array(process_status_column)
transition_column = np.array(transition_column)

num_rows = len(range(len(transition_column)))

current_department_id_column = np.array([np.random.choice([1, 4, 105]) for i in range(num_rows)])
ORDER_STATUS_id_column = np.array([i for i in range(num_rows)])

previous_department_id_column = [None]
for i in range(1, num_rows):
    if process_status_column[i] == 1 and process_status_column[i-1] != (1 or 2):
        previous_department_id_column.append(None)
    else:
        j = 1
        while current_department_id_column[i] == current_department_id_column[i-j]:
            j += 1
        previous_department_id_column.append(current_department_id_column[i-j])

comp = 0
for i in range(num_rows):
    if comp:
        if current_department_id_column[i] == current_department_id_column[i-1]:
            previous_department_id_column[i] = None
        else:
            comp = 0
    if previous_department_id_column[i] == None:
        comp = 1

department_change_column = ["(null)"]
for i in range(1, num_rows):
    if previous_department_id_column[i] == None and previous_department_id_column[i-1] != None:
        department_change_column.append("(null)")
    elif previous_department_id_column[i] != previous_department_id_column[i-1]:
        department_change_column.append("yes")
    else:
        department_change_column.append("no")

num_complaints = sum(calls_per_customer)

start_date = datetime.datetime.today() - datetime.timedelta(hours = 2)
dates = list(np.random.choice(range(0, 365), num_complaints + 30, replace = True))
dates.sort()
dates = [int(dates[i]) for i in range(len(dates))]
dates = [start_date + datetime.timedelta(days = dates[i]) for i in range(len(dates))]

from_time_column = [None] * num_rows
from_time_column[0]= dates[0]
j = 1
for i in range(1, num_rows):
    if previous_department_id_column[i] == None and previous_department_id_column[i-1] != None:
        from_time_column[i] = dates[j] + random.random() * datetime.timedelta(hours = int(np.random.choice(range(1, 3))))
        j += 1

for i in range(1, num_rows):
    if from_time_column[i] == None:
        from_time_column[i] = from_time_column[i-1] + (np.random.exponential(0.3) * datetime.timedelta(hours = 1))

until_time_column = from_time_column.copy()
until_time_column.pop(0)
until_time_column.append(until_time_column[-1]  + (np.random.exponential(0.3) * datetime.timedelta(hours = 1)))

insert_date_column = from_time_column.copy()
modified_timestamp_column = until_time_column.copy()

for i in range(num_rows):
    insert_date_column[i] += np.random.exponential(0.2) * datetime.timedelta(minutes = 1)
    modified_timestamp_column[i] += np.random.exponential(0.2) * datetime.timedelta(minutes = 1)
    if previous_department_id_column[i] == None:
        previous_department_id_column[i] = "(null)"

# Users
users_1 = list(range(4))
users_4 = list(range(4, 8))
users_105 = list(range(8, 12))

user = np.random.choice(eval(f"users_{current_department_id_column[0]}"))
user_id_column = [user]
for i in range(1, num_rows):
    if current_department_id_column[i] != current_department_id_column[i-1]:
        num = current_department_id_column[i]
        user = np.random.choice(eval(f"users_{num}"))
    user_id_column.append(user)


d = {"CURRENT_DEPARTMENT_ID": current_department_id_column, "ID": ORDER_STATUS_id_column,
     "PROCESS_STATUS": process_status_column, "SERVICE_ORDER_ID": service_order_id_column,
     "PREVIOUS_DEPARTMENT_ID": previous_department_id_column, 
     "TRANSITION": transition_column, "FROM_TIME": from_time_column,
     "UNTIL_TIME": until_time_column, "DEPARTMENT_CHANGE": department_change_column, 
     "INSERT_DATE": insert_date_column, "MODIFIED_TIMESTAMP_COLUMN": modified_timestamp_column,
     "USER": user_id_column, "CUSTOMER_CONTACT_ID": customer_contact_id_column,
     "PRIORITY": priority_column, "TYP": typ_column,
     "VEHICLE_LICENSE_PLATE": vehicle_license_plate_column, "QUALITY_LEVEL": quality_level_column}

df = pd.DataFrame(data = d)

df = df.sort_values(by = ["FROM_TIME"])


In [3]:
df_time = pd.DataFrame(columns=['SERVICE_ORDER_ID', 'DURATION', 'DURATION_NUM', 'START', 'END', 'TYPE', 'CUSTOMER_CONTACT_ID'])

for i in df["SERVICE_ORDER_ID"].unique():
  current = df.loc[df["SERVICE_ORDER_ID"] == i]
  start = current["FROM_TIME"].min()
  end = current["FROM_TIME"].max()
  duration = end - start
  customer_id = current["CUSTOMER_CONTACT_ID"].iloc[0]
  license_plate = current["VEHICLE_LICENSE_PLATE"].iloc[0]
  type = current["TYP"].iloc[0]
  df_time = df_time.append({'SERVICE_ORDER_ID': i, 'DURATION': duration, 'START': start, 'END': end, 'TYPE':type, 'CUSTOMER_CONTACT_ID': customer_id, 'VEHICLE_LICENSE_PLATE': license_plate}, ignore_index=True)

df_time["DURATION_NUM"] = (df_time["DURATION"].astype('timedelta64[s]') / 60)

In [4]:
###KPI1: "Average time until completion:"
print(str(df_time["DURATION"].mean()))

0 days 04:15:11.384204315


In [5]:
#KPI2: "Median time until completion:" 
display(str(df_time["DURATION"].median()))

'0 days 03:38:05.482056'

In [6]:
#KPI3: Mean time until completion by customer:
print(df_time.groupby(by="CUSTOMER_CONTACT_ID")["DURATION_NUM"].mean().sort_values(ascending=False))

CUSTOMER_CONTACT_ID
98    789.491667
77    692.627778
70    502.466667
5     484.936364
30    477.466667
         ...    
99    150.908333
46    143.862500
95    140.411667
61    139.516667
22     98.716667
Name: DURATION_NUM, Length: 100, dtype: float64


In [7]:
#KPI4: Mean time until completion by license plate number:
print(df_time.groupby(by="VEHICLE_LICENSE_PLATE")["DURATION_NUM"].mean().sort_values(ascending=False))


VEHICLE_LICENSE_PLATE
22.0    403.817308
13.0    354.635556
16.0    316.677778
12.0    297.840000
6.0     297.047727
11.0    292.996212
7.0     287.593750
1.0     271.640000
17.0    269.307971
15.0    263.578571
19.0    239.065476
3.0     237.819444
9.0     233.905072
14.0    233.239167
5.0     232.897436
24.0    227.696154
8.0     224.813636
0.0     218.209140
20.0    218.096377
4.0     216.505051
23.0    216.025333
10.0    214.607333
18.0    211.391667
21.0    205.285000
2.0     182.857018
Name: DURATION_NUM, dtype: float64


In [8]:
print("Welcome to the Meta-Questionnaire! Please make sure your data has been put in the following format:")
print("")
print("Columns: Categories")
print("Rows: Observations")
print("")
print("If you have a Column with Dates, please Label it as 'Time'. Make sure the dates have the format YYYY-MM-DD or YYYY/MM/DD")

print("We will now ask you a few questions regarding the KPI")
print("")

Welcome to the Meta-Questionnaire! Please make sure your data has been put in the following format:

Columns: Categories
Rows: Observations

If you have a Column with Dates, please Label it as 'Time'. Make sure the dates have the format YYYY-MM-DD or YYYY/MM/DD
We will now ask you a few questions regarding the KPI



In [10]:
# New class: KPIs
class KPI:
    
    """
    name = n, pillar = p, trg = target, prf = performance, quality_dummy = qd, 
    dom_correlation = dc, strength_dom_correlation = sdc,
    """
    
    def __init__(self, n, p, trg, prf, qd = 0, dc = 0, sdc= 0):
        self.n = n
        self.p = p
        self.trg = trg
        self.prf = prf
        self.qd = qd
        self.dc = dc
        self.sdc = sdc
        
        
# creating a list of KPI objects
KPIs = []

n = input("What is the first KPI you want to visualize? ")
p = input("Which pillar does it belong to? (either write it out or use coding: 1 fin, 2 process, 3 customer, 4 Learning)")
trg = int(input("What is your performance goal for this KPI? "))
prf = int(input("How good were you performing the last time you measured? "))
time_fiha = input("Should your data be recorded on a predetermined time interval? (Yes/No)")
q = input("Do you think this KPI is correltated to at least one other KPI? ")
###Start automated Tests
time_check = "No"
flawless = "No"
if "Time" in df:
    df["Time"] = df['Time'].astype('datetime64[ns]')

    keys = randomlist = random.sample(range(1, (int(beg_row_num) - 1)), int((beg_row_num/10)))

    time_check = "Yes"

    time_lord = []
    for i in keys:
        diff = df["Time"][i] - df["Time"][i-1]
        time_lord.append(diff)

    hourish = 0
    dayish = 0
    weekish = 0
    yearish = 0

    for i in time_lord:
        if i < np.timedelta64(1,'D'):
            hourish += 1
        elif i < np.timedelta64(1,'W'):
            dayish += 1
        elif i < np.timedelta64(1,'M'):
            weekish += 1
        elif i < np.timedelta64(1,'Y'):
            yearish += 1
    votelist = [hourish, dayish,weekish,yearish]
    votelist = sorted(votelist, reverse= True)

    delim = votelist[0]
    if delim >= (len(keys)/2):
        time_check = "Yes"
    else: time_check = "No"

    full_keys = list(df.index)
    full_keys = full_keys[1:-1]

    flawless = "Yes"

    for i in full_keys:
        diff1 = df["Time"][i] - df["Time"][i-1]
        diff2 = df["Time"][i+1] - df["Time"][i]
        if diff1 == diff2:
            pass
        else: flawless =  "No"
        
duplicates = df.duplicated()
duplicates = duplicates[duplicates[:] == True]
dupl_nr = len(duplicates)

if dupl_nr > 0:
    is_dupl = "Yes"
else: is_dupl = "No"

#We will now drop rows based off missing values:

#Only drop rows where all values are missing
full_dropped = df.dropna(how = "all")
fulldroprows = 1 - ((df.shape[0] - full_dropped.shape[0]) / df.shape[0] )
fulldroprows = round(fulldroprows, 2)

#Only drop rows where some values are missing
dropped_thresh = df.dropna(thresh = 2)
threshdroppedrows = 1 - ((df.shape[0] - dropped_thresh.shape[0]) / df.shape[0])
threshdroppedrows = round(threshdroppedrows, 2)

#Drop all rows where any values are missing
dropped_any = df.dropna()
anydroppedrows = 1 - ((df.shape[0] - dropped_any.shape[0]) / df.shape[0])
anydroppedrows = round(anydroppedrows, 2)

if fulldroprows < 0.95:
    test1 = "F"
else: test1 = "P"
    
dupl_relative = (df.shape[0] - dupl_nr/ df.shape[0])

if dupl_relative < 0.95:
    test2 = "F"
else: test2 = "P"

if time_fiha == "Yes" and time_check == "No":
    test3 = "F"
else: test3 = "P"

test4 = "P"

if test1 == test2 ==test3 == test4:
    secure = 0
else: secure = 1


qd = secure 
###End automated tests

if q in ["Yes","yes","Yup","Y","y"]:
    dc = input("To which one? ")
    sdc = int(input("On a scale from 0-100: How strong do you think the correlation is? "))/100
    KPIs.append(KPI(n, p, prf, qd, dc, sdc))
else:
    KPIs.append(KPI(n, p, prf, qd))
    
q = input("Do you want to add other KPIs? ")
if q in ["Yes","yes","Yup","Y","y"]:
    i = 1
else:
    i = 0
    
while i == 1:    
    n = input("Which KPI do you want to visualize next? ")
    p = input("Which pillar does it belong to? (either write it out or use coding: 1 fin, 2 process, 3 customer, 4 Learning) ")
    trg = int(input("What is your performance goal for this KPI? "))
    prf = int(input("How good were you performing the last time you measured? "))
    YN = input("Do you think this KPI is correltated to at least one other KPI? ")
    if YN in ["Yes","yes","Yup","Y","y"]:
        dc = input("To which one? ")
        sdc = int(input("On a scale from 0-100: How strong do you think the correlation is? "))/100
        KPIs.append(KPI(n, p, prg, dc, sdc))
    else:
        KPIs.append(KPI(n, p, prg))
    q = input("Do you want to add other KPIs? ")
    if q in ["Yes","yes","Yup","Y","y"]:
        i = 1
    else:
        i = 0

What is the first KPI you want to visualize?  TZR

Which pillar does it belong to? (either write it out or use coding: 1 fin, 2 process, 3 customer, 4 Learning) 2

What is your performance goal for this KPI?  150

How good were you performing the last time you measured?  40

Should your data be recorded on a predetermined time interval? (Yes/No) No

Do you think this KPI is correltated to at least one other KPI?  No

Do you want to add other KPIs?  No